# Rolling Quarterly NDVI Anomalies

JBW version

Notebook is only compatible on the `NCI` as it uses Landsat Collection 2

## Background

Understanding how the vegetated landscape responds to longer-term environmental drivers such as the El Nino Southern Oscillation (ENSO) or climate change, requires the calculation of seasonal anomalies. Seasonal anomalies subtract the long-term seasonal mean from a time-series, thus removing seasonal variability and highlighting change related to longer-term drivers. 

## Description

This notebook will calculate the previous 3 month standardized anomaly relative to the current date.  The long-term seasonal climatologies for the vegetation index `NDVI` have been pre-calculated and are stored on disk. Given an AOI (NSW or subset), the script will calculate the previous 3-month mean NDVI and subtract the mean from the long-term climatology, resulting in a map of vegetation anomalies for your AOI.  Optionally, the script will output a geotiff of the result. 

**IMPORTANT NOTES:** 

* It is a convention to establish climatologies based on a 30-year time range to account for inter-annual and inter-decadal modes of climate variability (often 1980-2010). As the landsat archive only goes back to 1987, the climatologies here have been calculated using the date-range `1992 - 2008` (inclusive).  This is in line with BoM's AVHRR climatology but could be extended.  
* Currently, the pre-computed climatologies are only `NDVI mean` and `NDVI Standard Deviation` 
* The pre-computed climatologies do not include LS7 after the SLC failure (2003).

???
* `.yaml` files for running datacube stats to calculate vegetation climatologies are located here: `'/g/data/r78/cb3058/dea-notebooks/vegetation_anomlies/dcstats'`. There is a different .yaml file for each season, and a corresponding custom datacube-statistics python file for each season; these are located here: `'/g/data/r78/datacube_stats/custom_stats'` 
* The pre-computed climatologies are stored here: `/g/data/r78/cb3058/dea-notebooks/vegetation_anomalies/results`.  The script below will use this string location to grab the data, so shifting the climatology mosaics to another location will require editing the `anomalies.py` script in the `src` folder. 
* So far, climatolgies have been produced for the `Northern Murray-Darling Basin` for the `SON` and `JJA` seasons; the `DJF` season has a mosaic that is 2/3 complete (59 out of 90 albers tiles). The `NW Queensland` region has mosaicked climatologies for `JJA` and `DJF`, but these are also only 2/3 complete (60 out of 90 albers tiles processed).

## Technical details

* **Products used:** ls5_nbart_albers, ls7_nbart_albers, ls8_nbart_albers
* **Analyses used:** NDVI, MSAVI, seasonal anomalies

## Getting Started

To run this analysis, go to the `Analysis Parameters` section and enter the relevant details, then run all the cells in the notebook. If running the analysis multiple times, only run the `Set up dask cluster` and `import libraries` cells once.

## Import libraries

In [3]:
import xarray as xr
from datacube.helpers import write_geotiff
import matplotlib.pyplot as plt
from dask.distributed import Client
import geopandas as gpd
import sys
import os
sys.path.append('src')
from anomalies import calculate_anomalies, load_landsat, display_map, map_shapefile
%load_ext autoreload
%autoreload 2

### Set up local dask cluster

Dask will create a local cluster of cpus for running this analysis in parallel. If you'd like to see what the dask cluster is doing, click on the hyperlink that prints after you run the cell and you can watch the cluster run.

In [ ]:
#delete old client if one still exists
client = locals().get('client', None)
if client is not None:
    client.close()
    del client
    
client = Client(n_workers=3, threads_per_worker=1, memory_limit='9GB')
client

## Analysis Parameters

The following cell sets the parameters, which define the area of interest and the season to conduct the analysis over. The parameters are:

* `veg_index`: The vegetation index to use for the analysis, either `'msavi'` or `'ndvi'`.
* `from_shape`: If providing a shapefile to the define the area of interest, set this parameter to `True`, otherwise set to `False`.
* `shp_fpath`: If you set `from_shape` to True, provide a filepath to the shapefile.
* `lat`, `lon`: If not using a shapefile to define the AOI, then use a latitide and longitude point. This point will be the centre point of your AOI.
* `buffer`: The length, in decimal degrees, either side of the `lat` and `lon` point that will define the AOI box. Keep this below `0.5` to avoid long load times
* `year`: The year of interest, e.g. `'2018'`
* `season`:  The season of interest, must be one of `'DJF'`, `'MAM'`, `'JJA'`, or `'SON'`
* `region`: This variable is to account for the climatology datasets being cacluated only for specific regions at this time.  Must be one of `'NMDB'` (for the Northern Murray Darling Basin) or `'NWQLD'` for NW Queensland

In [5]:
#input parameters
veg_index='ndvi'
from_shape = False
# shp_fpath = "data/nmdb_individual_catchments/CONDAMINE-CULGOA RIVERS.shp"
lat, lon = -33.2, 149.1
buffer = 0.15
year = '2018'
season = 'JJA'
region = 'NMDB'

#dask chunk size, shouldn't need to change
chunk_size = 750

### Examine your area of interest

In [ ]:
# If you're using a shapefile, run this cell
map_shapefile(gpd.read_file(shp_fpath))

In [6]:
# If your specifying a lat, lon and buffer, run this cell
display_map(y=(lat-buffer, lat + buffer), x=(lon-buffer, lon + buffer))

## Calculate the anomaly for the AOI

For large queries (e.g > 4000 x 4000 pixels), the code will take several minutes to run.  Queries much larger than ~15,000 x 15,000 pixels will start to fail due to memory limitations (that said, a 19,000 x 15,000 pixel catchment has been successfully run on the VDI). Check the x,y dimensions in the lazily loaded output to get idea of how big your result will be before you run  the `.compute()` cell.

In [ ]:
#Lazily run calculations, this will check for errors before
# we actually compute the results
anomalies = calculate_anomalies(veg_index=veg_index,
                                from_shape=from_shape,
                                shp_fpath=shp_fpath,
                                lat=lat,
                                lon=lon,
                                buffer=buffer,
                                year=year,
                                season=season,
                                region=region,
                                chunk_size=chunk_size)

print(anomalies)

In [ ]:
#this cell will actually compute the above calculations
anomalies = anomalies.compute()

## Export geotiff

In [ ]:
# Write geotiff to a location
write_geotiff(veg_index+"_"+year+"_"+season+ '_anomalies.tif', anomalies)

## Plot the result

If your AOI is very large, plotting the result can crash the notebook. In this case, its better to export the geotiff and view it in QGIS or ArcGIS

In [ ]:
if veg_index == 'msavi':
    anomalies.msavi_anomalies.plot(figsize=(15,15), vmin=-0.25, vmax=0.25, cmap='BrBG')
if veg_index == 'ndvi':
    anomalies.ndvi_anomalies.plot(figsize=(15,15), vmin=-0.25, vmax=0.25, cmap='BrBG')

plt.title(season+ ", " +year)
plt.show()